Now, coming to Week 2 task. For each of the authors, you ideally want to know more about their background, topics of interest, twitter links etc.

One easy way to grab it is going their muckrack journalist profile page and web scrapping it from there (refer to chapter 2, beautifulsoup portion to know how to do it).

Two ways to do it: first, you can search through the muckrack sitemap CSV file for near duplicate matches in similar or same names and scrape their profiles. 

Second method is letting Bing/Google do the job aka take the author name and outlet name and search Google programmatically using Specrom APIs. For example if you want to know more about Dave Sebastian WSJ.com just use it as a search key and find muckrack profile and proceed from there. 

How about you trying doing do both approaches on top 10-30 authors and see which one works best?

https://algorithmia.com/algorithms/specrom/Google_scraper

https://algorithmia.com/algorithms/specrom/Bing_scraper

In [6]:
import pickle
import pandas as pd
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import json
import time

In [7]:
with open('news.pkl', 'rb') as f:
    news = pickle.load(f)

In [12]:
news

,author,site_name,title,article_count,title_sentiment
0,Aja Styles,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,14.0,-9.09
1,Jake Johnson,Truthout,Congress Passes COVID Relief With Billions in ...,33.0,18.18
2,Christine Favocci,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,19.0,-38.46
3,William Rivers Pitt,Truthout,What Will Trump Attempt in His Last Days? We M...,14.0,0.00
4,Amy Goodman,Truthout,The Insufficient COVID Stimulus Must Not Be Fo...,19.0,-20.00
...,...,...,...,...,...
121126,Simon Bajkowski,Manchester Evening News,How Man City should line up vs Fulham in the P...,87.0,0.00
121127,Victoria Jones,WalesOnline,How to teach saving and spending to kids as yo...,92.0,0.00
121128,Victoria Jones,WalesOnline,Space experiment could unlock resources for mi...,92.0,0.00
121129,Nisha Mal,WalesOnline,Woman's home is in Tier 2 while her garden fal...,66.0,-10.00


In [10]:
s = 'fsfsgsfWR#^^$&%*^IULIP(_-9--))'
punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
for x in s.lower(): 
        if x in punctuations: s = s.replace(x, "") 
print(s)

fsfsgsfWRIULIP-9--


In [40]:
data = news.copy()
data = data[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False).reset_index()
columns = ['verified', 'interests', 'title', 'location', 'description', 
           'seen_in', 'website', 'twitter', 'linkedin', 'facebook', 'instagram']
for col in columns:
    data[col] = 0

In [41]:
data

,author,article_count,verified,interests,title,location,description,seen_in,website,twitter,linkedin,facebook,instagram
0,Neil Shaw,523,0,0,0,0,0,0,0,0,0,0,0
1,James Rodger,512,0,0,0,0,0,0,0,0,0,0,0
2,Jack Davis,251,0,0,0,0,0,0,0,0,0,0,0
3,Adam Wells,223,0,0,0,0,0,0,0,0,0,0,0
4,Sophie McCoid,218,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,Emma Dunkley,10,0,0,0,0,0,0,0,0,0,0,0
3887,Tanya Jefferies,10,0,0,0,0,0,0,0,0,0,0,0
3888,Liam Kelly,10,0,0,0,0,0,0,0,0,0,0,0
3889,Levi Sumagaysay,10,0,0,0,0,0,0,0,0,0,0,0


In [42]:
def get_top(n):
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index().head(n)
    urls = []
    for i in range(len(temp)):
        url = re.sub(r'(\w*.\w*@\w*.\w*)', '', temp.loc[i].author.lower()) # remove emails
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        url = re.sub(r'[^\\w-]', '', url) # remove punctuation
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(sa)(\s+)(\w+)(\s+)', '', url) # SA editors
        url = re.sub(r'(tulsa)(\s+)(world)', '', url) # tulsa world editors
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        # print(muckrack[muckrack.url.str.contains(url, regex=False)])
        urls.append(url)
    temp['Muckrack'] = urls
    return temp

get_top(300)

,author,article_count,Muckrack
0,Neil Shaw,523,https://muckrack.com/w
1,James Rodger,512,https://muckrack.com/
2,Jack Davis,251,https://muckrack.com/
3,Adam Wells,223,https://muckrack.com/w
4,Sophie McCoid,218,https://muckrack.com/
...,...,...,...
295,Luke Kenton,77,https://muckrack.com/
296,Peter Sblendorio,77,https://muckrack.com/
297,Emily Hodgkin,77,https://muckrack.com/
298,Tom Towers,76,https://muckrack.com/w


In [43]:
client = Algorithmia.client('simKLMHlWDMzWd+QcNRqVhI104r1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [12]:
data = get_top(3891)

In [ ]:
for j in range(len(data)):
    try:
        url = data.iloc[j].Muckrack
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
    except:
        print("wrong url", data.iloc[j].Muckrack, data.iloc[j].author)
        query = {"query": f"{data.iloc[j].author} site: muckrack"}
        url = algo.pipe(query).result
        if url:
            url = url[0]['url']
            print("right url", url)
            print()
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            sauce = urlopen(req).read()
    soup = bs.BeautifulSoup(sauce,'lxml')

wrong url https://muckrack.com/matthew-cooper Matthew Cooper
wrong url https://muckrack.com/svar-nanansen Svar Nanan-Sen
wrong url https://muckrack.com/seamus-duff Seamus Duff
wrong url https://muckrack.com/alessandra-scotto-di-santolo Alessandra Scotto di Santolo
wrong url https://muckrack.com/ryan-taylor Ryan Taylor
wrong url https://muckrack.com/beth-mishlerelmore Beth Mishler-Elmore
wrong url https://muckrack.com/liam-doyle Liam Doyle
wrong url https://muckrack.com/jessica-dolcourt Jessica Dolcourt
wrong url https://muckrack.com/unzela-khan Unzela Khan
wrong url https://muckrack.com/elle-may-rice Elle May Rice
wrong url https://muckrack.com/austin-boyd Austin Boyd
wrong url https://muckrack.com/molly-pike Molly Pike
wrong url https://muckrack.com/bryony-jewell Bryony Jewell
wrong url https://muckrack.com/lucas-hillpaul Lucas Hill-Paul
wrong url https://muckrack.com/saraaisha-kent Sara-Aisha Kent
wrong url https://muckrack.com/charlie-spiering Charlie Spiering
wrong url https://muck

In [8]:
def extract_data(link):
    info = {}
    info['name'] = data.iloc[j].author
    if soup.find_all(class_ = 'profile-verified'):
        info['verified'] = True
    if soup.find(class_ = 'person-details-item person-details-beats'):
        info['interests'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-tasks icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'person-details-item person-details-title'):
        info['title'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-building icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'person-details-item person-details-location'):
        info['location'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-map-marker icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'mr-font-size-lg mr-font-family-2 top-sm'):
        info['description'] = soup.find(class_ = 'mr-font-size-lg mr-font-family-2 top-sm').text.strip()
    info['seen_in'] = list(set([i.text for i in soup.find_all('a') if '/media-outlet/' in i.attrs['href']]))
    if soup.find(class_ = 'profile-section-social'):
        for i in soup.find(class_ = 'profile-section-social'):
            temp = [j[5:] for j in str(i).split() if 'href' in j]
            if temp: 
                temp = temp[0].strip('"')
                if 't.co' in temp:
                    info['website'] = temp
                if 'twitter' in temp:
                    info['twitter'] = temp
                if 'linkedin' in temp:
                    info['linkedin'] = temp
                if 'facebook' in temp:
                    info['facebook'] = temp
                if 'instagram' in temp:
                    info['instagram'] = temp